In [1]:
"""
Run exploration with high resolution within the connection strengths (100x100), but with only two different values for
the frequencies of the fast and slow oscillators.

"""

import numpy as np
from neurolib.optimize.exploration import BoxSearch
from neurolib.utils.functions import getPowerSpectrum
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.models.multimodel import MultiModel
from hopf_cfc_network import SlowFastHopfNetwork
from cfcUtils import modulation_index_general, mean_vector_length, phase_locking_value, mutual_information
from scipy.signal import find_peaks
import neurolib.utils.pypetUtils as pu
import neurolib.optimize.exploration.explorationUtils as eu
import matplotlib.pyplot as plt
import h5py

# w instrinsic
# 0.003 ~ 0.5Hz
# 0.005 ~ 0.8Hz
# 0.01 ~ 2Hz
# 0.05 ~ 8Hz
# 0.06 ~ 10Hz
# 0.08 ~ 12Hz
# 0.1 ~ 17Hz
# 0.2 ~ 32Hz
# 0.3 ~ 50Hz

In [2]:
DURATION = 20.0 * 1000  # ms
DT = 0.1  # ms

model = MultiModel(
    SlowFastHopfNetwork(
        number_of_slow_fast_units=1,
        slow_to_fast_connection=0.0,
        fast_to_slow_connection=0.0,
    )
)

model.params["duration"] = DURATION
model.params["sampling_dt"] = DT
model.params["dt"] = 0.01  # ms - lower dt for numba backend
# numba backend is the way to go in exploration - much faster
model.params["backend"] = "numba"

# manually add params you want to change during exploration, btw model.params is just a dictionary, so you can add whatever :)
model.params["slow_to_fast"] = 0.0
model.params["fast_to_slow"] = 0.0
model.params["bifurcation_param_slow"] = 0.25
model.params["bifurcation_param_fast"] = 0.25
model.params["frequency_slow"] = 0.04
model.params["frequency_fast"] = 0.2

In [3]:
parameters = ParameterSpace(
    {
        "slow_to_fast": np.linspace(0.0, 2., 15),
        "fast_to_slow": np.linspace(0.0, 2., 15),
        "frequency_slow": np.array([0.02, 0.04, 0.08]),
        "frequency_fast": np.array([0.15, 0.25, 0.35]),
        "bifurcation_param_slow": np.array([0.25, 0.6, 1.3, 4, 8]),
        "bifurcation_param_fast": np.array([0.25, 0.4, 0.6, 1, 4]),
    },
    allow_star_notation=True,
    kind="grid",
)
# Default params:
# slow_frequency: 0.04
# fast_frequency: 0.2
# bifurcation param: 0.25

In [4]:
def evaluateSimulation(traj):
    # get model with parameters for this run
    model = search.getModelFromTraj(traj)
    # extract stuff you want
    s_f_conn = model.params["slow_to_fast"]
    f_s_conn = model.params["fast_to_slow"]

    model.params["*connectivity"] = np.array([[0.0, f_s_conn], [s_f_conn, 0.0]])
    model.params['SlowFastHopfNet.SlowHopf_0.HopfMass_0.a'] = model.params["bifurcation_param_slow"]
    model.params['SlowFastHopfNet.FastHopf_1.HopfMass_0.a'] = model.params["bifurcation_param_fast"]
    model.params['SlowFastHopfNet.SlowHopf_0.HopfMass_0.w'] = model.params["frequency_slow"]
    model.params['SlowFastHopfNet.FastHopf_1.HopfMass_0.w'] = model.params["frequency_fast"]

    model.run()

    freq_slow, pow_slow = getPowerSpectrum(model.x.T[:, 0], dt=0.1, maxfr=60, spectrum_windowsize=1)
    freq_fast, pow_fast = getPowerSpectrum(model.x.T[:, 1], dt=0.1, maxfr=60, spectrum_windowsize=1)

    peaks_fast, _ = find_peaks(pow_fast, height=max(1e-3, 1.0 * np.std(pow_fast)))
    peaks_slow, _ = find_peaks(pow_slow, height=max(1e-3, 0.5 * np.std(pow_slow)))

    if (np.allclose(np.asarray(model.x.T[100000:150000, 0]), np.asarray(model.x.T[150000:, 0]), rtol=1e-3) and
            (np.allclose(np.asarray(model.x.T[100000:150000, 1]), np.asarray(model.x.T[150000:, 1]), rtol=1e-3))):

        # that is, if in the last 10 seconds of the simulation are in a constant value, then we
        # are in a fixed point, there's no oscillation --> we're not interested
        mi = []
        mvl_abs = []
        plv_abs = []
        mvl_angle = []
        plv_angle = []
        minfo = []

    else:
        phase_slow = np.arctan2(model.y[0, :], model.x[0, :])
        phase_fast = np.arctan2(model.y[1, :], model.x[1, :])
        amp_fast = np.sqrt(model.x[1, :] ** 2 + model.y[1, :] ** 2)

        mi = modulation_index_general(amp_fast, phase_slow, n_bins = 18)
        mvl = mean_vector_length(amp_fast, phase_slow)
        mvl_abs = np.absolute(mvl)
        mvl_angle = np.angle(mvl)
        plv = phase_locking_value(phase_fast, phase_slow)
        plv_abs = np.absolute(plv)
        plv_angle = np.angle(plv)
        minfo = mutual_information(phase_fast, phase_slow, bins=16, log2=False)


    result_dict = {
        "peaks_freq_fast": peaks_fast,
        "peaks_freq_slow": peaks_slow,
        "modulation_index": mi,
        "mean_vector_length_abs": mvl_abs,
        "mean_vector_length_angle": mvl_angle,
        "phase_locking_value_abs": plv_abs,
        "phase_locking_value_angle": plv_angle,
        "mutual_information": minfo,
    }


    search.saveToPypet(result_dict, traj)


In [5]:
search = BoxSearch(
    model=model,
    evalFunction=evaluateSimulation,
    parameterSpace=parameters,
    filename="medium_resolution_exploration.hdf",
    ncores=50,
)

MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `./data/hdf/medium_resolution_exploration.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Number of parameter configurations: 50625
MainProcess root INFO     BoxSearch: Environment initialized.


In [6]:
search.loadResults()

MainProcess root INFO     Loading results from ./data/hdf/medium_resolution_exploration.hdf


OSError: Unable to open file (truncated file: eof = 943942938, sblock->base_addr = 0, stored_eof = 943958138)

In [ ]:
search.dfResults

In [ ]:
def try_division(row, exceptVal):
    try:
        return row["slow_to_fast"]/row["fast_to_slow"]
    except ZeroDivisionError:
        return np.inf
def check_zero_length(row):
    try:
        if len(row)==0:
            row = np.nan
    except TypeError:
        pass
    return row

In [ ]:
search.dfResults['max_freq_slow'] = search.dfResults.apply(
    lambda row: np.amax(row["peaks_freq_slow"],initial=0), axis=1
    )
search.dfResults['max_freq_fast'] = search.dfResults.apply(
    lambda row: np.amax(row["peaks_freq_fast"], initial=0), axis=1
    )
search.dfResults["ratio_freq"]=search.dfResults.apply(
    lambda row: row["frequency_fast"]/row["frequency_slow"],axis=1)

search.dfResults["ratio_conn"]=search.dfResults.apply(
    lambda row: try_division(row,0),axis=1)

search.dfResults['n_peaks_freq_slow'] = search.dfResults.apply(
    lambda row: len(row["peaks_freq_slow"]), axis=1
    )
search.dfResults['n_peaks_freq_fast'] = search.dfResults.apply(
    lambda row: len(row["peaks_freq_fast"]), axis=1
    )


In [ ]:
search.dfResults['modulation_index'] = search.dfResults.apply(
    lambda row: check_zero_length(row["modulation_index"]), axis=1)
    

In [ ]:
search.dfResults['mean_vector_length_angle'] = search.dfResults.apply( 
    lambda row: check_zero_length(row["mean_vector_length_angle"]), axis=1)
search.dfResults['mean_vector_length_abs'] = search.dfResults.apply( 
    lambda row: check_zero_length(row["mean_vector_length_abs"]), axis=1)
search.dfResults['phase_locking_value_angle'] = search.dfResults.apply( 
    lambda row: check_zero_length(row["phase_locking_value_angle"]), axis=1) 
search.dfResults['phase_locking_value_abs'] = search.dfResults.apply( 
    lambda row: check_zero_length(row["phase_locking_value_abs"]), axis=1) 
search.dfResults['mutual_information'] = search.dfResults.apply(
    lambda row: check_zero_length(row["mutual_information"]), axis=1)

In [ ]:
search.dfResults

In [ ]:
search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow]

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="modulation_index",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="Modulation index for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "modulation_index_medium_expl_bifurcation_params_ratios_conn.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="modulation_index",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Modulation index",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "modulation_index_medium_expl_ratios_freq.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="mutual_information",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Mutual Information",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mutual_information_medium_expl_ratios_freq.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="mean_vector_length_abs",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Mean vector length Absolute",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mean_vector_length_abs_medium_expl_ratios_freq.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="mean_vector_length_angle",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Mean vector length Angle",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mean_vector_length_angle_medium_expl_ratios_freq.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="phase_locking_value_abs",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Phase Locking Value Absolute",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "phase_locking_value_abs_medium_expl_ratios_freq.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults,
    par1=["slow_to_fast", r"slow$\to$fast"],
    par2=["fast_to_slow", r"fast$\to$slow"],
    plot_key="phase_locking_value_angle",
    nan_to_zero=True,
    by= "ratio_freq",
    one_figure= True,
    plot_key_label="Phase Locking Value Angle",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "phase_locking_value_angle_medium_expl_ratios_freq.png",
)

In [ ]:
search.dfResults[search.dfResults.n_peaks_freq_slow >= 3]

In [ ]:
search.dfResults[search.dfResults.n_peaks_freq_fast >= 3]

In [ ]:
eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="mutual_information",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="mutual information for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mutual_information_medium_expl_bifurcation_params_ratios_conn.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="phase_locking_value_angle",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="phase_locking_value_angle for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "phase_locking_value_angle_medium_expl_bifurcation_params_ratios_conn.png",
)


In [ ]:
eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="phase_locking_value_abs",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="phase_locking_value_abs for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "phase_locking_value_abs_medium_expl_bifurcation_params_ratios_conn.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="mean_vector_length_angle",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="mean_vector_length_angle for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mean_vector_length_angle_medium_expl_bifurcation_params_ratios_conn.png",
)

In [ ]:
eu.plotExplorationResults(
    search.dfResults[search.dfResults.slow_to_fast>search.dfResults.fast_to_slow],
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="mean_vector_length_abs",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="mean_vector_length_abs for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "mean_vector_length_abs_medium_expl_bifurcation_params_ratios_conn.png",
)

In [ ]:
#search.dfResults.groupby(["ratio_freq"]).agg(np.nanmean)
gb = search.dfResults.groupby('ratio_freq')
df1,df2,df3,df4,df5,df6,df7,df8,df9 = [gb.get_group(x) for x in gb.groups]
warnings.resetwarnings()

In [ ]:
df_temp = search.dfResults.groupby(["ratio_freq","ratio_conn","bifurcation_param_fast","bifurcation_param_slow"],as_index=False).agg(np.nanmean)

In [ ]:
df_temp[df_temp.slow_to_fast > df_temp.fast_to_slow]

In [ ]:
for i, df in df_temp[df_temp.slow_to_fast > df_temp.fast_to_slow].groupby(by="ratio_conn"):
    df_pivot = df.pivot_table(values="mean_vector_length_abs", index="bifurcation_param_fast", columns="bifurcation_param_slow")
    df_pivot.fillna(0)
    plt.imshow(df_pivot)
    plt.xlabel("bifurcation_param_slow")
    plt.ylabel("bifurcation_param_fast")
    plt.xlim(np.min(df.bifurcation_param_slow),np.max(df.bifurcation_param_slow))
    plt.ylim(np.max(df.bifurcation_param_fast),np.min(df.bifurcation_param_fast))
    plt.title(f"Ratio of conn{np.unique(np.asarray(df.ratio_conn))}")
    plt.colorbar()
    ax = plt.gca()
    ax.invert_yaxis()
    plt.show()

In [ ]:
eu.plotExplorationResults(
    df_temp,
    par1=["bifurcation_param_slow", r"Bifurcation Param Slow"],
    par2=["bifurcation_param_fast", r"Bifurcation Param Fast"],
    plot_key="mean_vector_length_abs",
    nan_to_zero=True,
    by= "ratio_conn",
    one_figure= True,
    plot_key_label="mean_vector_length_abs for r = slow_to_fast/fast_to_slow",
    contour=["max_freq_slow", "max_freq_fast"],
    contour_color=[["white"], ["blue"]],
    contour_levels=[[6,12,32], [6,12,32]],
    contour_alpha=[1.0, 1.0],
    contour_kwargs={
        0: {"linewidths": (2,), "clabel": True},
        1: {"linewidths": (2,), "clabel": True},
    },
    savename = "test.png",
)